In [136]:
import os
import zipfile
import time
import pickle
import gc

import pandas as pd
import numpy as np
from tqdm import tqdm

from utils import load_pickle, dump_pickle, get_feature_value, feature_spearmanr, feature_target_spearmanr, addCrossFeature, calibration
from utils import raw_data_path, feature_data_path, cache_pkl_path, analyse

from sklearn.metrics import log_loss
import lightgbm as lgb

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit

In [150]:
def CustomCV(data, k=1):
    train_days = 6 - k
    for i in range(k):
        train_start_day = 19 + i
        train_end_day = 19 + i + train_days - 1
        test_day = 19 + i + train_days
        fold_index_train = data[(data.day >= train_start_day) & (data.day <= train_end_day)].index
        fold_index_test = data[data.day == test_day].index
        yield fold_index_train, fold_index_test
        
        
def CustomCV2(data, k=1):
    fold_index_train = data[(data.day >= 19) & (data.day <= 22)].index
    fold_index_test = data[data.day == 23].index
    yield fold_index_train, fold_index_test
    
    fold_index_train = data[(data.day >= 19) & (data.day <= 23)].index
    fold_index_test = data[data.day == 24].index
    yield fold_index_train, fold_index_test
    
    
def CustomCV_19_21_22(data, k=1):
    fold_index_train = data[(data.day >= 19) & (data.day <= 21)].index
    fold_index_test = data[data.day == 22].index
    yield fold_index_train, fold_index_test
    
    
def CustomCV_19_22_23(data, k=1):
    fold_index_train = data[(data.day >= 19) & (data.day <= 22)].index
    fold_index_test = data[data.day == 23].index
    yield fold_index_train, fold_index_test
    
def CustomCV_20_23_24(data, k=1):
    fold_index_train = data[(data.day >= 20) & (data.day <= 23)].index
    fold_index_test = data[data.day == 24].index
    yield fold_index_train, fold_index_test
    
    
def CustomCV_19_23_24(data, k=1):
    fold_index_train = data[(data.day >= 19) & (data.day <= 23)].index
    fold_index_test = data[data.day == 24].index
    yield fold_index_train, fold_index_test

In [223]:
all_data_path = feature_data_path + 'all_data_all_features_until_25.pkl'
all_data = load_pickle(all_data_path)

target = 'is_trade'

features = load_pickle(feature_data_path + 'features_0420.pkl')

# features = list(all_data.columns)
# features.remove('is_trade')
# features.remove('instance_id')

len(features)

273

In [225]:
if __name__ == '__main__':


    eval_data = all_data[all_data.day == 24]
    eval_set = [(eval_data[features], eval_data[target])]

    lgb_clf = lgb.LGBMClassifier(objective='binary', device='gpu',  n_jobs=-1, silent=False)

#     # 参数的组合
    lgb_param_grad = {'n_estimators': (2000, ),
                      'learning_rate': (0.02, ),

                      'max_depth': (4,),
                      'num_leaves': (20, ),
                      'min_child_samples': (200, ),
#                       'min_child_weight': (0.1, ),
#                       'min_split_gain': (0.1, ),
                      

#                       'colsample_bytree': (1.0, ),
                      'subsample': (0.7, ),
#                       'subsample_freq': (1, ),
                      
#                       'reg_alpha': (0,),
                      'reg_lambda': (10, ),
                      
                      'max_bin': (63, ),
                      
                      'gpu_use_dp': (False, ),
                      }

 # 参数的组合
#     lgb_param_grad = {'n_estimators': (2000, ),
#                       'learning_rate': (0.02, ),

#                       'max_depth': (5,),
#                       'num_leaves': (18,),
#                       'min_child_samples': (100, ),
#                       'min_child_weight': (0.1, ),
#                       'min_split_gain': (0.1,),
                      

#                       'colsample_bytree': (1.0, ),
#                       'subsample': (0.7, ),
#                       'subsample_freq': (1,),
                      
# #                       'reg_alpha': (0,),
#                       'reg_lambda': (0.02, ),
                      
#                       'max_bin': (60, 63, 66),
                      
#                       'gpu_use_dp': (False, ),
#                       }

    clf = GridSearchCV(lgb_clf, param_grid=lgb_param_grad, scoring='neg_log_loss',
                       cv=CustomCV_19_23_24(all_data, k=1), n_jobs=-1, verbose=1, refit=False, return_train_score=True)

    clf.fit(all_data[features], all_data[target],
            feature_name=features,
#             categorical_feature=nominal_feats,
            early_stopping_rounds=200, eval_set=eval_set, eval_names='test_day_24', verbose=50
           )

    print('=====')
    print("Best parameters set found on development set:")
    print(clf.best_params_)

    print('=====')
    print("Best parameters set found on development set:")
    print(clf.best_score_)


Fitting 1 folds for each of 1 candidates, totalling 1 fits
Training until validation scores don't improve for 200 rounds.
[50]	test_day_24's binary_logloss: 0.240578
[100]	test_day_24's binary_logloss: 0.12586
[150]	test_day_24's binary_logloss: 0.0923848
[200]	test_day_24's binary_logloss: 0.0828559
[250]	test_day_24's binary_logloss: 0.0800433
[300]	test_day_24's binary_logloss: 0.0791067
[350]	test_day_24's binary_logloss: 0.0786669
[400]	test_day_24's binary_logloss: 0.0784383
[450]	test_day_24's binary_logloss: 0.0782907
[500]	test_day_24's binary_logloss: 0.0782005
[550]	test_day_24's binary_logloss: 0.0781371
[600]	test_day_24's binary_logloss: 0.0780751
[650]	test_day_24's binary_logloss: 0.0780243
[700]	test_day_24's binary_logloss: 0.0779925
[750]	test_day_24's binary_logloss: 0.0779614
[800]	test_day_24's binary_logloss: 0.0779307
[850]	test_day_24's binary_logloss: 0.0779183
[900]	test_day_24's binary_logloss: 0.0779067
[950]	test_day_24's binary_logloss: 0.0778959
[1000]	t

[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  2.2min finished


=====
Best parameters set found on development set:
{'gpu_use_dp': False, 'learning_rate': 0.02, 'max_bin': 63, 'max_depth': 4, 'min_child_samples': 200, 'n_estimators': 2000, 'num_leaves': 20, 'reg_lambda': 10, 'subsample': 0.7}
=====
Best parameters set found on development set:
-0.0778861058324


In [222]:
pd.DataFrame(data=clf.cv_results_)[['rank_test_score', 'mean_test_score', 'mean_train_score', 'param_reg_lambda', 'param_min_child_weight',
                                    'param_min_child_samples', 'param_num_leaves', 'param_subsample', 'param_colsample_bytree', 
                                    'param_min_split_gain','param_subsample_freq',
                                    'param_max_bin', 'param_gpu_use_dp']]






,rank_test_score,mean_test_score,mean_train_score,param_reg_lambda,param_min_child_weight,param_min_child_samples,param_num_leaves,param_subsample,param_colsample_bytree,param_min_split_gain,param_subsample_freq,param_max_bin,param_gpu_use_dp
0,3,-0.078070,-0.076863,0.02,0.1,100,18,0.7,1,0.1,1,60,False
1,1,-0.077896,-0.076735,0.02,0.1,100,18,0.7,1,0.1,1,63,False
2,2,-0.077982,-0.076444,0.02,0.1,100,18,0.7,1,0.1,1,66,False
